In [1]:
import numpy as np
import pandas as pd
from scipy.stats import bernoulli as brnl
import os

In [732]:
class Lab1:
    def __init__(self, var_path, var):
        prob_filename = f'prob_{str(var)}.csv'
        table_filename = f'table_{str(var)}.csv'
        self.prob_path = os.path.join(var_path, prob_filename)
        self.table_path = os.path.join(var_path, table_filename)
        self.prob = pd.read_csv(self.prob_path, header=None)
        self.table = pd.read_csv(self.table_path, header=None)
    
    
    def calc_PC(self):
        PC = np.zeros(self.table.shape[0])
        for i in range(self.table.shape[0]):
            X, Y = np.where(self.table == i)
            for x, y in zip(X, Y):
                PC[i] += self.prob.iloc[0, y] * self.prob.iloc[1, x]
        return np.around(PC, 4)
    
    
    def calc_PMC(self):
        PMC = np.zeros((self.table.shape[0], self.table.shape[0]))
        for i in range(self.table.shape[0]):
            X, Y = np.where(self.table == i)
            for x, y in zip(X, Y):
                PMC[y][i] += self.prob.iloc[0, y] * self.prob.iloc[1, x]
        return np.around(PMC, 4)
    
    
    def calc_PM_C(self, PC, PMC):
        PC = PC.T
        PM_C = np.zeros((PMC.shape[0], PMC.shape[0]))
        for i in range(PMC.shape[0]):
            PM_C[i] = PMC[i] / PC
        return np.around(PM_C, 4)
    
    
    def calc_Bayes(self, PC, PM_C):
        temp = np.zeros(PC.shape[0], dtype=np.int64)
        loss = 0
#         print(PC.shape)
#         PC_T = PC.T
#         print(PC.shape)
        PM_C_T = PM_C.T
        for i in range(PM_C_T.shape[0]):
            temp[i] = int(np.where(PM_C_T[i] == PM_C_T[i].max())[0][0])
#             print(type(temp[i]))
            print(f'M[{temp[i]}] ===> C[{i}]')
        print(temp)
        for i in range(PC.shape[0]):
#             print(temp[i])
            loss += PC[i] * (1 - PM_C[int(temp[i])][i])
        print(f'Loss: {loss}')
#         return temp

    
    def calc_stochastic(self, PC, PM_C):
        loss = 0
        PM_C_T = PM_C.T

        temp1 = np.zeros((PC.shape[0], PC.shape[0]))
        temp2 = np.zeros(PC.shape[0])
        temp3 = np.zeros_like(temp1)

        for i in range(PC.shape[0]):
            indexes = np.where(PM_C_T[i] == PM_C_T[i].max())[0]
#             print(indexes, len(indexes))
            for index in indexes:
                temp1[i][index] = 1 / len(indexes)
            temp2[i] = len(indexes)
            print("Количество максимумов в столбце [",i,"] =", len(indexes))
#         print(temp1)


#         for i in range(PC.shape[0]):
#             t = temp2[i]
#             index = np.where(temp1[i] != 0)
#             print(index)
#             for index in indexes:
#                 print(index)
#             temp3[i][index] = brnl.rvs(1 / t)
#             print(i)
        
#             print(np.where(temp3[i] != 0))
#         print(temp3)
        for i in range(PC.shape[0]):
            t = temp2[i]
            for j in range(PC.shape[0]):
                if temp1[i][j] != 0:
                    temp3[i][j] = brnl.rvs(1 / t)
                    t -= 1
                if temp3[i][j]:
                    break
        for i in range(PC.shape[0]):
            print(f'M[{np.where(temp3[i] != 0)[0][0]}] ===> C[{i}]')
        print(temp3)
        for i in range(20):
            l1 = 0
            for j in range(20):
                l1 += PM_C_T[i][j] * temp1[i][j]
            loss += (1 - l1) * PC[i]
        print(f'Loss: {loss}')
        
        
    
    def __repr__(self):
        return 'Lab1 is not so clear :('

In [733]:
a = Lab1('vars', '01')

In [734]:
PC = a.calc_PC()
print('Распределение шифртекста:')
pd.DataFrame(PC)

Распределение шифртекста:


,0
0,0.04
1,0.05
2,0.06
3,0.05
4,0.04
5,0.05
6,0.04
7,0.06
8,0.05
9,0.04


In [735]:
assert PC.sum().round(10) == 1.0, 'Should be 1.0'

In [736]:
PMC = a.calc_PMC()
print('Распеделение открытых текстов и шифртекстов:')
pd.DataFrame(PMC)

Распеделение открытых текстов и шифртекстов:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000,0.012,0.024,0.012,0.000,0.012,0.000,0.024,0.012,0.000,0.012,0.000,0.012,0.048,0.012,0.024,0.012,0.012,0.012,0.000
1,0.004,0.000,0.002,0.000,0.004,0.004,0.000,0.000,0.004,0.000,0.002,0.000,0.000,0.002,0.006,0.000,0.004,0.004,0.002,0.002
2,0.000,0.002,0.000,0.006,0.006,0.000,0.002,0.002,0.002,0.006,0.002,0.002,0.002,0.000,0.004,0.000,0.000,0.002,0.002,0.000
3,0.002,0.004,0.000,0.000,0.002,0.002,0.000,0.002,0.002,0.004,0.002,0.004,0.006,0.000,0.000,0.004,0.000,0.000,0.002,0.004
4,0.002,0.002,0.004,0.008,0.000,0.002,0.002,0.000,0.002,0.000,0.004,0.002,0.004,0.000,0.002,0.002,0.000,0.004,0.000,0.000
5,0.002,0.004,0.004,0.000,0.002,0.000,0.004,0.004,0.002,0.000,0.000,0.002,0.002,0.002,0.000,0.002,0.006,0.000,0.002,0.002
6,0.002,0.004,0.002,0.000,0.002,0.002,0.000,0.000,0.000,0.002,0.000,0.004,0.000,0.006,0.000,0.004,0.006,0.004,0.002,0.000
7,0.002,0.000,0.000,0.000,0.000,0.004,0.006,0.000,0.002,0.006,0.000,0.000,0.004,0.004,0.000,0.004,0.000,0.002,0.000,0.006
8,0.002,0.002,0.004,0.000,0.000,0.004,0.000,0.000,0.000,0.000,0.006,0.002,0.000,0.000,0.004,0.006,0.002,0.000,0.004,0.004
9,0.002,0.008,0.000,0.002,0.004,0.000,0.004,0.002,0.000,0.000,0.000,0.002,0.000,0.004,0.002,0.004,0.002,0.000,0.002,0.002


In [737]:
assert PMC.sum().round(10) == 1.0, 'Should be 1.0'

In [738]:
PM_C = a.calc_PM_C(PC, PMC)
print('Условное распределение:')
pd.DataFrame(PM_C)

Условное распределение:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.00,0.24,0.4000,0.24,0.00,0.24,0.00,0.4000,0.24,0.00,0.24,0.00,0.24,0.600,0.24,0.4000,0.24,0.24,0.24,0.00
1,0.10,0.00,0.0333,0.00,0.10,0.08,0.00,0.0000,0.08,0.00,0.04,0.00,0.00,0.025,0.12,0.0000,0.08,0.08,0.04,0.05
2,0.00,0.04,0.0000,0.12,0.15,0.00,0.05,0.0333,0.04,0.15,0.04,0.05,0.04,0.000,0.08,0.0000,0.00,0.04,0.04,0.00
3,0.05,0.08,0.0000,0.00,0.05,0.04,0.00,0.0333,0.04,0.10,0.04,0.10,0.12,0.000,0.00,0.0667,0.00,0.00,0.04,0.10
4,0.05,0.04,0.0667,0.16,0.00,0.04,0.05,0.0000,0.04,0.00,0.08,0.05,0.08,0.000,0.04,0.0333,0.00,0.08,0.00,0.00
5,0.05,0.08,0.0667,0.00,0.05,0.00,0.10,0.0667,0.04,0.00,0.00,0.05,0.04,0.025,0.00,0.0333,0.12,0.00,0.04,0.05
6,0.05,0.08,0.0333,0.00,0.05,0.04,0.00,0.0000,0.00,0.05,0.00,0.10,0.00,0.075,0.00,0.0667,0.12,0.08,0.04,0.00
7,0.05,0.00,0.0000,0.00,0.00,0.08,0.15,0.0000,0.04,0.15,0.00,0.00,0.08,0.050,0.00,0.0667,0.00,0.04,0.00,0.15
8,0.05,0.04,0.0667,0.00,0.00,0.08,0.00,0.0000,0.00,0.00,0.12,0.05,0.00,0.000,0.08,0.1000,0.04,0.00,0.08,0.10
9,0.05,0.16,0.0000,0.04,0.10,0.00,0.10,0.0333,0.00,0.00,0.00,0.05,0.00,0.050,0.04,0.0667,0.04,0.00,0.04,0.05


In [739]:
assert PM_C.sum().round(3) == 20.0, 'Should be 20.0'

In [740]:
print('Детерминистическая решающая функция и потери:')
a.calc_Bayes(PC, PM_C)

Детерминистическая решающая функция и потери:
M[17] ===> C[0]
M[0] ===> C[1]
M[0] ===> C[2]
M[0] ===> C[3]
M[2] ===> C[4]
M[0] ===> C[5]
M[7] ===> C[6]
M[0] ===> C[7]
M[0] ===> C[8]
M[2] ===> C[9]
M[0] ===> C[10]
M[16] ===> C[11]
M[0] ===> C[12]
M[0] ===> C[13]
M[0] ===> C[14]
M[0] ===> C[15]
M[0] ===> C[16]
M[0] ===> C[17]
M[0] ===> C[18]
M[7] ===> C[19]
[17  0  0  0  2  0  7  0  0  2  0 16  0  0  0  0  0  0  0  7]
Loss: 0.7240000000000003


In [741]:
print('Стохастическая функция и потери:')
a.calc_stochastic(PC, PM_C)

Стохастическая функция и потери:
Количество максимумов в столбце [ 0 ] = 1
Количество максимумов в столбце [ 1 ] = 1
Количество максимумов в столбце [ 2 ] = 1
Количество максимумов в столбце [ 3 ] = 1
Количество максимумов в столбце [ 4 ] = 2
Количество максимумов в столбце [ 5 ] = 1
Количество максимумов в столбце [ 6 ] = 2
Количество максимумов в столбце [ 7 ] = 1
Количество максимумов в столбце [ 8 ] = 1
Количество максимумов в столбце [ 9 ] = 2
Количество максимумов в столбце [ 10 ] = 1
Количество максимумов в столбце [ 11 ] = 1
Количество максимумов в столбце [ 12 ] = 1
Количество максимумов в столбце [ 13 ] = 1
Количество максимумов в столбце [ 14 ] = 1
Количество максимумов в столбце [ 15 ] = 1
Количество максимумов в столбце [ 16 ] = 1
Количество максимумов в столбце [ 17 ] = 1
Количество максимумов в столбце [ 18 ] = 1
Количество максимумов в столбце [ 19 ] = 1
M[17] ===> C[0]
M[0] ===> C[1]
M[0] ===> C[2]
M[0] ===> C[3]
M[2] ===> C[4]
M[0] ===> C[5]
M[7] ===> C[6]
M[0] ===> C